# Downloading data

In [1]:
from sklearn.datasets import fetch_20newsgroups
train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')

# Preprocessing

In [2]:
def extract(d, keys):
    return dict((k, d[k]) for k in keys if k in d)

Naive bayes is computationally really complex, so in order to fast up the training (and test) time i reduced the dataset size

In [3]:
import pandas as pd
train_data=extract(train_data,["data","target"])
train_data=pd.DataFrame.from_dict(train_data)
train_data=train_data.sample(frac=1,random_state=69)
train_X=train_data['data'].tolist()
train_y=train_data['target'].tolist()

test_data=extract(test_data,["data","target"])
test_data=pd.DataFrame.from_dict(test_data)
#test_data=test_data.sample(1000,random_state=69)
test_X=test_data['data'].tolist()
test_y=test_data['target'].tolist()

print("Number of train examples: ", len(train_X))
print("Number of test exaples: ", len(test_X))

Number of train examples:  11314
Number of test exaples:  7532


Define a method useful for printing out all the metrics useful to determine the quality of the classification

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

def evaluate(y_test, y_pred):
  print("accuracy:", accuracy_score(y_test, y_pred))
  print("precision:", precision_score(y_test, y_pred, average='micro'))
  print("recall:", recall_score(y_test, y_pred, average='micro'))
  print("")

Connection to drive, necessary to acces to a couple of files like the vocabulary and the list of stopwords, necessary for the naive bayes algorithm

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The files mentioned before are downloaded and transformed to a vector of words

In [6]:
vocabulary_path="/content/drive/My Drive/Colab Notebooks/NaiveBayes/google-10000-english.txt"
#vocabulary_path="/content/drive/My Drive/Colab Notebooks/NaiveBayes/english-3000.txt"
vocabulary_file=open(vocabulary_path,"r")
vocabulary_text = vocabulary_file.read()
vocabulary=vocabulary_text.split()
vocabulary_file.close()

stopwords_path="/content/drive/My Drive/Colab Notebooks/NaiveBayes/english-stopwords.txt"
stopwords_file=open(stopwords_path,"r")
stopwords_text = stopwords_file.read()
stopwords=stopwords_text.split()
stopwords_file.close()

# Naive Bayes algorithm implementation

In [7]:
import numpy as np
import string
import sys

class NaiveBayesTextClassifier:
  class_priors={}
  word_given_class={}

  vocabulary=[]

  def __init__(self,vocabulary, stopwords):
    self.vocabulary=list(set(vocabulary)-set(stopwords))

  def train(self,X,y):
    document_by_class={}
    #Dividing documents by class  
    for i in range(0,len(y)):
      if y[i] not in document_by_class:
        document_by_class[y[i]]=[]
      document_by_class[y[i]].append(X[i])

    #Esteeming the probabilities
    i=1
    for classification, documents in document_by_class.items():
      sys.stdout.write("\r Calculating probability for classification number %i out of %i" % (i ,len(document_by_class.keys())))
      i=i+1
      sys.stdout.flush()
      #Esteeming the priors of each value
      self.class_priors[classification]=len(documents)/len(y)
      #Esteeming the conditional probabilities of each document given the class
      words_count=dict((i,0) for i in vocabulary)
      self.word_given_class[classification]={}
      #Creating an unique document made by the conjunction of all documents
      unique_document=" ".join(documents)
      #Preprocessing of the document
      unique_document=unique_document.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) #removing all punctuation
      unique_document=unique_document.lower() #Converting to lowercase all the characters of the document
      words=unique_document.split()
      #Estimation of the conditional probabilties
      for word in words:
        if word in vocabulary:
          words_count[word]=words_count[word]+1
      for word, n_occurences in words_count.items():
        self.word_given_class[classification][word]=(n_occurences+1)/(len(words)+len(vocabulary))

  def predict(self,document):
    best_class=None
    best_probability=None
    #Preprocessing of the document    
    temp_document=document.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) #removing all punctuation
    temp_document=temp_document.lower() #Converting to lowercase all the characters of the document
    words=temp_document.split()
    #Calculating the maximum likelihood for each classification
    for classification, word_probabilites in self.word_given_class.items():
      #Calculating such probabilities can bring really easilly to underflows due to the really small moltiplicated between each other
      #To avoid this problem instead of moltiplicating the probabilities we can sum the logarithms of each of them
      #The logarithm is in fact a monothone function so the results of the optimization problem doesn't change
      current_probability=np.log(self.class_priors[classification]) 
      for word in words:
        if word in self.vocabulary:
          current_probability=current_probability+np.log(self.word_given_class[classification][word])
      if best_probability==None or current_probability>best_probability:
        best_class=classification
        best_probability=current_probability
    return best_class

# Algorithm Training

In [10]:
classifier=NaiveBayesTextClassifier(vocabulary,stopwords)
classifier.train(train_X,train_y)

 Calculating probability for classification number 3 out of 20

KeyboardInterrupt: ignored

# Algorithm Test

In [9]:
predictions=[]
nTestExamples=len(test_X)
for i in range(0,nTestExamples):
  sys.stdout.write("\r Calculating example %i out of %i" % (i+1 ,nTestExamples))
  sys.stdout.flush()
  prediction=classifier.predict(test_X[i])
  predictions.append(prediction)
print()
predictions=[-1 if v is None else v for v in predictions]
evaluate(test_y[0:nTestExamples],predictions)

 Calculating example 7532 out of 7532
accuracy: 0.7294211364843335
precision: 0.7294211364843335
recall: 0.7294211364843335



# Conclusions

The results obtained are pretty decent, i think they could be improved even further by using a better vocabulary and a better list of stopwords